In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

import time
import tqdm

import pandas as pd

In [6]:
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

# URL base
url = "https://www.flexicar.es/porsche/segunda-mano/#/brands/bmw/brands/audi"
c_driver.get(f'{url}')
time.sleep(3)

In [7]:
# CARGAR WEB
scroll_pause = 1
scroll_step = 1000
max_scrolls = 200
scrolls = 0

while scrolls < max_scrolls:
    c_driver.execute_script(f"window.scrollBy(0, {scroll_step});")
    time.sleep(scroll_pause)
    scrolls += 1
    if scrolls % 10 == 0:
        print(f"{scrolls} / {max_scrolls}")

10 / 200
20 / 200
30 / 200
40 / 200
50 / 200
60 / 200
70 / 200
80 / 200
90 / 200
100 / 200
110 / 200
120 / 200
130 / 200
140 / 200
150 / 200
160 / 200
170 / 200
180 / 200
190 / 200
200 / 200


In [8]:
# Inicializar datos
data = []

xpath_cards = "/html/body/div[3]/div/main/div[2]/div/div[2]/div[3]/div"
cars = c_driver.find_elements(By.XPATH, xpath_cards)

for car in tqdm.tqdm(cars):
    car_class = car.get_attribute("class")
    if "MuiGrid-grid-sm-6" in car_class:
        car_img = car.find_element(By.XPATH, "./div/a/div[1]/div[1]/div/div/div/div[1]/img").get_attribute("src")
        car_name = car.find_element(By.XPATH, "./div/a/div[2]/div[1]/div[1]/h3").text
        car_price = car.find_element(By.XPATH, "./div/a/div[2]/p").text

        car_year = car.find_element(By.XPATH, "./div/a/div[2]/div[2]/ul/li[1]").text
        car_km = car.find_element(By.XPATH, "./div/a/div[2]/div[2]/ul/li[2]").text
        motor = car.find_element(By.XPATH, "./div/a/div[2]/div[1]/div[1]/p[2]").text

        data.append({
            'img': car_img,
            'name': car_name,
            'year': car_year,
            'price': car_price,
            'km': car_km,
            'motor': motor,
            'img': car_img
        })


100%|██████████| 1379/1379 [03:13<00:00,  7.13it/s]


In [9]:
csv = pd.DataFrame(data)
csv

,img,name,year,price,km,motor
0,https://www.flexicar.es/images/903000000147499...,BMW Serie 1,2021,19.490 €,85.977 km,118i
1,https://www.flexicar.es/images/903000000161152...,Audi A1,2017,16.490 €,48.836 km,Active Kit 1.4 TDI 66kW (90CV) Sportback
2,https://www.flexicar.es/images/903000000151458...,BMW Z4,2022,40.790 €,54.486 km,sDrive20i
3,https://www.flexicar.es/images/903000000150512...,BMW Serie 1,2020,21.590 €,61.215 km,118i Auto.
4,https://www.flexicar.es/images/903000000149232...,BMW Serie 1,2022,23.990 €,93.881 km,118d
...,...,...,...,...,...,...
1147,https://www.flexicar.es/images/903000000154636...,Audi A1,2014,11.490 €,112.511 km,1.2 TFSI 86cv Ambition
1148,https://www.flexicar.es/images/903000000157939...,Audi A3,2019,26.990 €,75.163 km,S line 35 TFSI 110kW S tronic Sportback
1149,https://www.flexicar.es/images/903000000157560...,BMW Serie 2,2018,22.490 €,115.000 km,218d
1150,https://www.flexicar.es/images/903000000157970...,Audi Q2,2017,17.990 €,162.327 km,sport ed 1.6 TDI 85kW (116CV) S tronic


In [10]:
csv.to_csv('../../../data/raw/flexicar.es.csv', index=False)